## 1-3. Number Theoretic Transform (NTT)
**GOAL:** faster multiplication between polynomials.

In previous section, we saw that following naive polynomial multiplication is slow.

We will improve it in this section using Foruier transformation and introduce its integer variant, NTT, very briefly. 

In [1]:
# Functions from previous lecturenote
import torch
import math
import cmath

stddev = 3.2
N = 2**10
Q = 2**27

def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

We can use Fourier transform to perform faster 'convolution'.
[Convolution theorem](https://en.wikipedia.org/wiki/Convolution_theorem) say that "the Fourier transform of a convolution of two functions (or signals) is the pointwise product of their Fourier transforms."

Complexity

A naive convolution has complexity of $O(n^2)$

FFT (fast Fourier transform) has complexity of $O(n \log n)$, and pointwise multiplication has complexity $O(n)$, thus total complexity is 
$$
O(n \log n + n ) = O(n \log n).
$$

If we consider the multiplication of polynomial as convolution of coefficient vector, it assumes $X^M = 1$ for some $M$.
However, we use a ring $\mathcal{R} = \mathbb{Z}[X]/\left< X^N+1 \right>$, where $X^N = -1.$

A easiest (but little bit inefficient) way to perform normal FFT is padding $N$ zeros as we have $X^{2N} = {-1}^2 = 1$.

PyTorch naturally supports FFT.

As FFT is defined over complex numbers, we map numbers in $\mathbb{Z}_Q$ to real numbers in $[0,1)$ before FFT simply using division by $Q$.
For secret keys, we don't need to do such transformation; as secret key is binary, we can consider multiplication of $\boldsymbol{z}$ as a *subset sum* of coefficients.

In [2]:

# @param: scale decides wether or not to map Z_Q to [0,1).
def polyfft(a, N, Q, scale=True):
    zeros = torch.zeros(N, dtype=torch.float64)

    apad = torch.cat((a, zeros))
    if scale:
        apad /= Q

    return torch.fft.fft(apad)

def polyifft(afft, N, Q):
    a = torch.fft.ifft(afft)
    aflip = torch.real(a[:N] - a[N:])
    aflip -= torch.round(aflip)

    aflip *= Q
    aint = aflip.to(torch.int32)
    
    aint %= Q

    return aint


Now we compare the results.
First, generate $\boldsymbol{a}$ and $\boldsymbol{z}$.

In [3]:
# secret key
z = keygen(N)

# random polynomial
a = uniform(N, Q)

a, z

(tensor([ 93289529,  96154302, 110129538,  ...,  65250308,  90388436,
             62944]),
 tensor([1, 0, 1,  ..., 0, 1, 0]))

In [4]:
# ordinary method
azslow = polymult(a, z, N, Q)
azslow

tensor([26585163, 72264883, 10091987,  ..., 16923841, 24083077, 13107663],
       dtype=torch.int32)

In [5]:
# using fft
A = polyfft(a, N, Q, scale=True)
Z = polyfft(z, N, Q, scale=False)

az = polyifft(A*Z, N, Q)

az

tensor([26585163, 72264883, 10091987,  ..., 16923841, 24083077, 13107662],
       dtype=torch.int32)

Same result, but the runtime differs a lot.

### 1.3.1. More efficient negacyclic FFT 
We map $f(x) \rightarrow f^*(x) = f(x) - x^Nf(x) = -f(x)(x^N-1)$ which and do a *cyclic* product modulu $2^{2N}-1$.
$f^*(x)$ is naturally mapped to $2f(x)$.

Multiplication of $f$ and $g$ can be found by 
$$
f^* g^* = -f(x^N+1)\cdot -g(x^N-1)
= 2(fg-x^Nfg),
$$
which naturally maps to $4fg$.

A better feature is that the first half of $f^*g^*$ is mapped to $2fg$, hence we don't need to fold the results.

For more detail, see [reference](https://jeremykun.com/2022/12/09/negacyclic-polynomial-multiplication/)

In [6]:
# @param: scale decides wether or not to map Z_Q to [0,1).
def polyfft_nofold(a, N, Q, scale=True):
    apad = torch.cat([a, -a])
    apad = apad.to(torch.float64)
    
    if scale:
        apad /= Q

    return torch.fft.fft(apad)

def polyifft_nofold(afft, N, Q):
    a = torch.fft.ifft(afft)

    areal = torch.real(a)
    areal = 0.5*areal
    areal = areal - torch.round(areal)
    areal *= Q
    aint = areal.to(torch.int32)
    
    aint %= Q

    return aint[:N]

In [7]:
# using fft
A = polyfft_nofold(a, N, Q, scale=True)
Z = polyfft_nofold(z, N, Q, scale=False)

az = polyifft_nofold(A*Z, N, Q)

az

tensor([26585163, 72264883, 10091987,  ..., 16923840, 24083077, 13107662],
       dtype=torch.int32)

In [8]:
az - azslow

tensor([ 0,  0,  0,  ..., -1,  0, -1], dtype=torch.int32)

### 1.3.2. More efficient negacyclic FFT with a "twist"

Looking at the result of NTT above, we can see that there are many zeros (especially, in all odd-indexed elements).


In [9]:
A

tensor([ 0.0000+0.0000j, 23.1473-657.7842j,  0.0000-0.0000j,
         ...,  2.7542+238.7902j,  0.0000+0.0000j,
        23.1473+657.7842j], dtype=torch.complex128)

Here, we investigate to reduce the operation by removing such redundant elements. 
We can do FFT of size N/2 to perform negacyclic multiplication!

We do a negacyclic FFT, for we convert the given vector of coefficients $\boldsymbol{a}$ to $\boldsymbol{b}$ of length N/2 where
$$
b_j = (a_j - i a_{N/2 + j}) w^j.
$$
Here, $w$ is a $2N$-th root of unity, $e^{−\pi i/N}$.
Then, we do FFT on $\boldsymbol{b}$. 

To multiply two polynomial, we perform pointwise multiplication of the FFTed values.

The inverse FFT for a given value $\boldsymbol{c} = FFT(\boldsymbol{b})$, is used to recover the product.
The product $\boldsymbol{a}$ is given as
$$
a_j = Real(b_jw^j) \text{ and } a_{N/2+j} = Imag(b_jw^j).
$$


See [nuFHE document](https://nufhe.readthedocs.io/en/latest/implementation_details.html?highlight=ntt#polynomial-multiplication) and [this blog](https://jeremykun.com/2022/12/09/negacyclic-polynomial-multiplication/) for detail

In [10]:
root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

In [11]:
def negacyclic_fft(a, N, Q, scale=True):
    acomplex = a.to(torch.complex128)
    
    if scale:
        acomplex /= Q

    a_precomp = (acomplex[:N//2] + 1j * acomplex[N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((torch.real(b), torch.imag(b)))
    a -= torch.round(a)

    a *= Q
    aint = a.to(torch.int32)
    aint %= Q

    return aint

In [12]:
# using fft
A = negacyclic_fft(a, N, Q, scale=True)
Z = negacyclic_fft(z, N, Q, scale=False)

az = negacyclic_ifft(A*Z, N, Q)

az

tensor([26585163, 72264883, 10091987,  ..., 16923840, 24083077, 13107663],
       dtype=torch.int32)

In [13]:
az - azslow

tensor([ 0,  0,  0,  ..., -1,  0,  0], dtype=torch.int32)

## Appendix. reducing mutiplication and division of Q

A muliplication between torus element is not defined.
Only external multiplication is allowed.
In other word, we can define and naturally extend it to polynomial
$$
\mathbb{T} \times \mathbb{Z} \mapsto \mathbb{T}
$$
but not
$$
\mathbb{T} \times \mathbb{T} \mapsto \mathbb{T}.
$$

Hence, the multiplicand should be integer (polynomial).

In [14]:
def negacyclic_fft(a, N, Q, scale=True):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[:N//2] + 1j * acomplex[N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((torch.real(b), torch.imag(b)))

    aint = torch.round(a).to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

In [15]:
# using fft
A = negacyclic_fft(a, N, Q, scale=True)
Z = negacyclic_fft(z, N, Q, scale=False)

az = negacyclic_ifft(A*Z, N, Q)

az

tensor([26585163, 72264883, 10091987,  ..., 16923841, 24083077, 13107663],
       dtype=torch.int32)

In [16]:
az - azslow

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

Replacing rounding to floor will introduce some error, but tolerable.

In [17]:
def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[:N//2] + 1j * acomplex[N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((torch.real(b), torch.imag(b)))

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

In [18]:
# using fft
A = negacyclic_fft(a, N, Q)
Z = negacyclic_fft(z, N, Q)

az = negacyclic_ifft(A*Z, N, Q)

az

tensor([26585164, 72264883, 10091987,  ..., 16923840, 24083077, 13107663],
       dtype=torch.int32)

In [19]:
az - azslow

tensor([ 1,  0,  0,  ..., -1,  0,  0], dtype=torch.int32)